### Load DIC data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic
from IPython.display import display
from matplotlib_scalebar.scalebar import ScaleBar
from skimage.filters import threshold_local, sobel, threshold_adaptive
import skimage.morphology as mph
from skimage import feature, measure
import scipy.ndimage as nd

from skimage.transform import rescale, resize, downscale_local_mean
from scipy import ndimage
import matplotlib as mpl

%matplotlib osx

DicFilePath = "/Users/mbcx9rt5/Dropbox (Research Group)/new davis/zrh/"

# SetA1 = hrdic.Map(DicFilePath, "SetA_001.TXT")
# SetA1.setPatternPath("SetA_opt.jpg", 4)
# SetA1.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)

# SetA2 = hrdic.Map(DicFilePath, "SetA_002.TXT")
# SetA2.setPatternPath("SetA_opt.jpg", 4)
# SetA2.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)

SetA3 = hrdic.Map(DicFilePath, "SetA_003.TXT")
SetA3.setPatternPath("SetA_opt.jpg", 4)
SetA3.setCrop(xMin=100,xMax=100,yMin=100,yMax=100)

EbsdMap = ebsd.Map("zrh/zr4H_a_clean2", "hexagonal")
EbsdMap.loadSlipSystems("defdap/slip_systems/hexagonal_withca.txt", cOverA=1.593)
EbsdMap.buildQuatArray()
EbsdMap.findPhaseBoundaries()
EbsdMap.findBoundaries(boundDef = 4)
EbsdMap.findGrains(minGrainSize= 100)
EbsdMap.calcGrainMisOri(calcAxis = False)
EbsdMap.calcAverageGrainSchmidFactors()

SetA3.homogPoints = np.array(( (384, 552), (718, 854), (530, 206), (1066, 774), (1188, 220), (73, 477), (152, 809), (140, 992), (383,1042),
                              (39,208), (458, 109), (853,345), (916, 160), (181,521), (685,554), (1037, 872), (885, 986)
                             ))
EbsdMap.homogPoints = np.array(( (460, 555), (705, 752), (584, 331), (976, 704), (1089, 356), (223, 492), (271, 711), (252, 835), (442,867),
                                (206,322), (533,268), (827,426), (881,308), (308, 524), (691,557), (952, 771), (832, 844)
                               ))

SetA3.linkEbsdMap(EbsdMap)
SetA3.findGrains(minGrainSize=50)

Loaded DaVis 8.4.0 data (dimensions: 1429 x 1253 pixels, sub-window size: 16 x 16 pixels)
Loaded EBSD data (dimensions: 1310 x 1054 pixels, step size: 0.1 um)


In [ ]:
if self.phaseArray[j, i]==-1 or self.phaseArray[j, i]==2 or self.phaseArray[j, i]==3 or phaseArrayShifted[j, i]==-1 or phaseArrayShifted[j, i]==2 or phaseArrayShifted[j, i]==3:
    self.boundaries[j, i] = 0

In [ ]:
SetA3.homogPoints = np.array(())
EbsdMap.homogPoints = np.array(())

In [ ]:
EbsdMap.setHomogPoint()
SetA3.setHomogPoint()

### Plot DIC

In [17]:
SetA3.plotMaxShear(plotGBs=True, vmax=0.1, dilateBoundaries=False)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, sharey=True, sharex=True)
ax1.imshow(EbsdMap.phaseArray)
ax2.imshow(-EbsdMap.boundaries)

### Fix bounds

In [14]:
plt.imshow(SetA3.crop(SetA3.max_shear),vmax=0.08)

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap2._init()
cmap2._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

boundariesImage = -SetA3.boundaries
#boundariesImage2 = mph.binary_dilation(boundariesImage)
plt.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)

boundariesImage3 = mph.binary_dilation(boundariesImage)
plt.imshow(boundariesImage3, cmap=cmap2, vmin=0, vmax=1)

### Find phases

In [2]:
EbsdMap.findPhaseBoundaries()

In [11]:
EbsdMap.plotPhaseBoundaryMap(dilate=True)

In [4]:
EbsdMap.plotPhaseMap()

In [ ]:
phase2 = mph.remove_small_objects(EbsdMap.phaseArray, min_size=5, in_place=False, connectivity=2)

phaseArrayShifted = np.full((EbsdMap.yDim, EbsdMap.xDim), -3)
phaseArrayShifted[:-1, :-1] = phase2[1:, 1:]

bound = np.zeros((EbsdMap.yDim, EbsdMap.xDim))
bound = np.where(np.not_equal(phase2, phaseArrayShifted), -1, 0)

plt.imshow(bound)

### Ward phase map to DIC

In [12]:
newmap = SetA3.warpToDicFrame(EbsdMap.phaseArray, order=0, preserve_range=True)

In [13]:
plt.imshow(newmap)

### Phase stats

In [15]:
zircstrains = []
hydridestrains = []
sppstrains = []

for x in range(100,SetA3.xDim-200):
    for y in range(100,SetA3.xDim-200):
        if newmap[y][x]==1:
            zircstrains.append(SetA3.crop(SetA3.max_shear)[y][x])
        if newmap[y][x]==2:
            hydridestrains.append(SetA3.crop(SetA3.max_shear)[y][x])
        if newmap[y][x]==3:
            sppstrains.append(SetA3.crop(SetA3.max_shear)[y][x])
            
zircstrains = np.asarray(zircstrains)
hydridestrains = np.asarray(hydridestrains)
sppstrains = np.asarray(sppstrains)

In [ ]:
def printStats(selMap,label='?'):
    print("{0}:    \t{1:.2f}\t{2:.2f}\t{3:.2f}\t{4:.2f}\t{5:.2f}\t{6:.2f}".format(label,np.min(selMap)*100,np.percentile((selMap)*100,0.1),np.mean(selMap)*100,np.percentile((selMap)*100,99.9),np.max(selMap)*100,np.percentile((selMap),99.9)/np.mean(selMap)))

print("Paramater(%) \tMin\tP0.1\tMean\tP99.9\tMax\tH")

printStats(SetA3.crop(SetA3.max_shear),label="All")
printStats(zircstrains,label="Zirc")
printStats(hydridestrains,label="Hydride")
printStats(sppstrains,label="SPP")

In [19]:
f, (ax1) = plt.subplots(1)
f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

counts, bins, bars = ax1.hist(zircstrains*100, label='metal',bins=np.linspace(0,15,200), color='r', histtype='step', normed=True)
counts, bins, bars = ax1.hist(hydridestrains*100, label='hydride',bins=np.linspace(0,15,200), color='b', histtype='step', normed=True)
#counts, bins, bars = ax1.hist(sppstrains, label='spp',bins=np.linspace(0,0.15,200), color='g', histtype='step', normed=True)

plt.xlabel("Effective shear strain (%)",{'size': 12})
plt.ylabel("Frequency",{'size': 12})
plt.legend()

center = (bins[:-1] + bins[1:]) / 2

plt.show()

f.savefig("test.pdf", bbox_inches='tight')

### Warp, remove small

In [ ]:
newmap3 = SetA3.warpToDicFrame(EbsdMap.phaseArray, order=1)

In [ ]:
newmap3[newmap3<0.0116]=5 #Non-indexed
newmap3[newmap3<0.019]=4 #Zirc was 0.157
newmap3[newmap3<0.0199]=3 #Hydride
newmap3[newmap3<1]=2 #SPP

newmap3[newmap3==5]=0 #Non-indexed
newmap3[newmap3==4]=0 #Zirc
newmap3[newmap3==3]=1 #Hydride
newmap3[newmap3==2]=0 #SPP

plt.imshow(newmap3)

In [ ]:
boolmap = newmap3.astype(bool)

remsml = mph.remove_small_objects(boolmap, min_size=10, in_place=False, connectivity=1)

phaseArrayShifted = np.full((SetA3.yDim, SetA3.xDim), -3)
phaseArrayShifted[:-1, :-1] = remsml[1:, 1:]

# where shifted array not equal to starting array, set to -1
newmap2 = np.zeros((SetA3.yDim, SetA3.xDim))
newmap2 = np.where(np.not_equal(remsml, phaseArrayShifted), 1, 0)


### Plot

In [ ]:
import skimage.morphology as mph

plt.imshow(SetA3.crop(SetA3.max_shear),vmax=0.08)

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap2._init()
cmap2._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

boundariesImage = newmap2
#boundariesImage = mph.binary_dilation(boundariesImage)
#boundariesImage=mph.binary_closing(boundariesImage)
plt.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)

#boundariesImage3 = mph.binary_dilation(boundariesImage)
#plt.imshow(boundariesImage3, cmap=cmap2, vmin=0, vmax=1)

### Export boundaries

In [ ]:
#EXPORT BOUNDARY TO FILE
from skimage import morphology as mph
import matplotlib as mpl
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
boundariesImage = newmap2
plt.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)

import scipy.misc
scipy.misc.imsave('newdata/a_phase.bmp', boundariesImage)

In [ ]:
###IMPORT FROM FILE

import scipy

phaseboundaries = np.zeros((SetA3.yDim, SetA3.xDim))
phaseboundaries = scipy.misc.imread('newdata/a_phase.bmp').astype(int)
phaseboundaries[phaseboundaries < 200] = 0

phaseboundaries[phaseboundaries > 200] = 1

### Plot

In [ ]:
import skimage.morphology as mph

plt.imshow(SetA3.crop(SetA3.max_shear),vmax=0.08)
plt.colorbar()

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap2._init()
cmap2._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

phaseboundaries = mph.binary_dilation(phaseboundaries)
grainboundaries = mph.binary_dilation(-SetA3.boundaries)
plt.imshow(-SetA3.boundaries, cmap=cmap2, vmin=0, vmax=1)
plt.imshow(phaseboundaries, cmap=cmap1, vmin=0, vmax=1)

### Tweak boundary

### Select DIC map

In [ ]:
DicMap=SetA3

print("Max Exx: {0:.2f}%".format(np.max(DicMap.crop(DicMap.f11))*100))
print("Max shear: {0:.2f}%".format(np.max(DicMap.crop(DicMap.max_shear))*100))
print("Min shear: {0:.2f}%".format(np.min(DicMap.crop(DicMap.max_shear))*100))
print("Mean Exx: {0:.2f}%".format(np.mean(DicMap.crop(DicMap.f11))*100))
print("Mean shear: {0:.2f}%".format(np.mean(DicMap.crop(DicMap.max_shear))*100))
print("Points: {0}".format(len(DicMap.crop(DicMap.max_shear)[0])*len(DicMap.crop(DicMap.max_shear)[1])))

### Load and threshold BSE image

In [ ]:
from skimage.transform import rescale
image = plt.imread(DicMap.patternImPath)
#image = nd.filters.median_filter(image,size=16)
image2 = rescale(image, 1.0 / 4.0)

In [ ]:
plt.imshow(DicMap.crop(image2),cmap='binary')

In [ ]:
#thres = threshold_adaptive(image2, 1501, offset=10)

thresh = threshold_local(image2, 1501, offset=10)
image3 = image2 > thresh


In [ ]:
plt.imshow(DicMap.crop(image3),cmap='binary')

In [ ]:
binary = rescale(thres, 1.0 / 4.0)


binary = mph.binary_closing(binary)
binary=mph.remove_small_objects(binary,min_size=2000,connectivity=4)

In [ ]:
plt.figure(figsize=(16,10))
plt.imshow(binary,cmap='binary')
plt.show()

In [ ]:
edges = sobel(binary)
edges[edges>=0.01] = 1
edges[edges<0.01] = 0

In [ ]:
plt.figure(figsize=(16,10))
plt.imshow(edges)
plt.show()

### Plot DIC and skeleton

In [ ]:
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1,cmap1.N + 3)

In [ ]:
fig = plt.figure(frameon=False,figsize=(8,7))
ax = plt.Axes(fig, [0., 0., 0.9, 1.])
#fig,ax=plt.subplots(1, 1, figsize=(10, 8),frameon=False)
ax.set_axis_off()
fig.add_axes(ax)

img = ax.imshow(DicMap.crop(DicMap.max_shear)*100,vmax=10)
ax.imshow(DicMap.crop(edges),cmap=cmap1)

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

scalebar = ScaleBar(0.078125e-6) # 1 pixel = 0.2 meter
plt.gca().add_artist(scalebar)

cbar=plt.colorbar(img, ax=ax,fraction=0.03, pad=0.02, anchor=(0.0, 0.22))
cbar.set_label(label="Effective shear strain (%)",size=12)
cbar.ax.tick_params(labelsize=11) 
plt.show()

fig.savefig('fig2.png',dpi=300, bbox_inches='tight', pad_inches=0)

##### 